# Asset Type 2

## Examples, Samples, Use cases:

* Unique artifacts on Ledger, e.g. art pieces
* Fiat currency with unique number in digital form.
* ...

## Attributes:

* Transferable ✅
* Quantifiable ❌


## Tests:

### Initialisation:

The initialisation is for initialise the python scripts in the notebook get it connected to CorDapp. \
Expected to see a _base directory_ path.

In [1]:
import os
import sys
import math
import time
import IPython
import logging
import warnings
import datetime
import pandas as pd
import numpy  as np
import webbrowser
import pprint as pp
from random   import randrange
from notebook import notebookapp
from IPython.display import display
from IPython.core.display import HTML

# Initialise the notebook settings
def init_notebook():
    sys.path.insert(1, os.path.join(sys.path[0], '{}/lib/'.format(os.environ['BaseDir'])))
    display(HTML("<style>.container { width:98% !important; }</style>"))
    %matplotlib inline
    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Initialise the Corda nodes
def init_cordaNodes():
    from corda5Interface import Corda5
    apiServer='https://localhost:8888/api/v1/flow'
    apiServerAuth=('admin', 'admin')
    package = 'com.r3.developers.configurableInstrument'
    replace_maters_list = { 'API_SERVER': apiServer, 'PACKAGE': package}

    nodes = {
            'Authority' : {'x500': 'CN=Authority, OU=Test Dept, O=R3, L=London, C=GB' ,'hash': 'F93770F20D7A' },
            'Bob'       : {'x500': 'CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'       ,'hash': '53B3F5FCD707' },
            'Charlie'   : {'x500': 'CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'   ,'hash': '597B2CEE8DB5' },
            'Alice'     : {'x500': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'     ,'hash': '49717D1246AF' },
            }

    h = Corda5(level=logging.INFO, nodes=nodes, apiServer=apiServer, apiServerAuth=apiServerAuth, version=2)

    log = h.log
    log.info("Base directory {}".format(os.environ['BaseDir']))
    
    return h

init_notebook()
h = init_cordaNodes()

 19-05 10:02:20 | I | Base directory /Users/Weicong.Su/Downloads/0519/configurableLedgerAsset/python3


### Issue:

* Issuer gets to decide the name, transferable or not, expiry or not, redeemable or not, and can add multiple/additional attributes (HashMap) on the same State/Asset.
* Applications like art pieces and Fiat currency with unique number, will be transferable, and redeemable (cash the principal on maturity) but not quantifiable

In Asset type 2, all the issuing should be succeed, with **Transferable** and **Non-Quantifiable** assets.

In [2]:
# Issue Example 1 -- Unique artifacts on Ledger -- Transferable, Non-Quantifiable Instrument
# Expected -- succeed

(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Unique artifact. The Mona Lisa', 
                                              quantity=None, transferable=True, expiry=None, verifiable=True, 
                                              attributes={'holding authority': 'The Louvre Museum, Paris'})

h.message([req_id, response, return_val_uuid])

 19-05 10:06:09 | I | Issuing instrument='Unique artifact. The Mona Lisa' to 'Alice'
 19-05 10:06:09 | I | Running action 'Issue' on 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:06:58 | I | ad3ebdb1-60a5-4d5c-956f-63cb5473ad47
 19-05 10:06:58 | I | Time taken = 0:00:48.494439


7221.1684461969.663759<Response [200]>ad3ebdb1-60a5-4d5c-956f-63cb5473ad47

In [4]:
# Issue Example 2 -- Fiat currency with unique number in digital form. -- Transferable, Non-Quantifiable Instrument
# Expected -- succeed

(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Digital Currency INR 500 Note',
                                              quantity=None, transferable=True, expiry=None, verifiable=True, 
                                              attributes={'number': 'OMV 336048', 'year': '1981'})

h.message([req_id, response, return_val_uuid])

 19-05 10:07:26 | I | Issuing instrument='Digital Currency INR 500 Note' to 'Alice'
 19-05 10:07:26 | I | Running action 'Issue' on 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:08:10 | I | b367c236-10cb-4f76-836a-74e997c48971
 19-05 10:08:10 | I | Time taken = 0:00:44.275260


7221.1684462046.497303<Response [200]>b367c236-10cb-4f76-836a-74e997c48971

### Query:

Query is used for query the instruments that one/all held.

In [5]:
# Query Exmaple 1 -- Query all the instruments

h.query_all_nodes()

 19-05 10:08:17 | I | Result for query=ListInstrument from 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.072169)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,ad3ebdb1-60a5-4d5c-956f-63cb5473ad47,Unique artifact. The Mona Lisa,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'holding authority': 'The Louvre Museum, Paris'}"
1,b367c236-10cb-4f76-836a-74e997c48971,Digital Currency INR 500 Note,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'number': 'OMV 336048', 'year': '1981'}"


 19-05 10:08:22 | I | Result for query=ListInstrument from 'Bob/53B3F5FCD707/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.069403)


""


 19-05 10:08:26 | I | Result for query=ListInstrument from 'Charlie/597B2CEE8DB5/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.068104)


""


 19-05 10:08:30 | I | Result for query=ListInstrument from 'Alice/49717D1246AF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.070433)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,ad3ebdb1-60a5-4d5c-956f-63cb5473ad47,Unique artifact. The Mona Lisa,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'holding authority': 'The Louvre Museum, Paris'}"
1,b367c236-10cb-4f76-836a-74e997c48971,Digital Currency INR 500 Note,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'number': 'OMV 336048', 'year': '1981'}"


In [6]:
# Query Exmaple 2 -- Query Alice's instruments

h.query('Alice')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,ad3ebdb1-60a5-4d5c-956f-63cb5473ad47,Unique artifact. The Mona Lisa,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'holding authority': 'The Louvre Museum, Paris'}"
1,b367c236-10cb-4f76-836a-74e997c48971,Digital Currency INR 500 Note,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'number': 'OMV 336048', 'year': '1981'}"


### Transfer:

* Not all instruments are transferable.
* If transferable -- will work on the basic nature of the instrument decided right from the issue point(by the issuer).
* Check = not expired apart from being owned. 

In Asset type 2 (Transferable, Non-Quantifiable Instrument):
* **Full** transfer from a **holder** to a holder should be **succeed**.
* **Partial** transfer from a **holder** to a holder should be **failed**.
* Any transfer from a **issuer** to a holder should be **failed**.
* Any transfer from a **non holder** to a holder should be **failed**.

In [7]:
# Transfer Example 1 -- Partial transfer from Alice to Bob
# Expected -- succeed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Bob')

h.message([req_id, response, return_val])

 19-05 10:10:15 | I | Tranfering instrument id 'b367c236-10cb-4f76-836a-74e997c48971' to 'Bob'
 19-05 10:10:15 | I | Running action 'Transfer' on 'Alice/49717D1246AF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:11:11 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@233cf089
 19-05 10:11:11 | I | Time taken = 0:00:56.324007


7221.1684462215.640028<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@233cf089

In [8]:
# Transfer Example 2 -- Full transfer from Alice to Charlie
# Expected -- succeed

(req_id, response, return_val) = h.transfer('ad3ebdb1-60a5-4d5c-956f-63cb5473ad47', 'Alice', 'Charlie')

h.message([req_id, response, return_val])

 19-05 10:11:21 | I | Tranfering instrument id 'ad3ebdb1-60a5-4d5c-956f-63cb5473ad47' to 'Charlie'
 19-05 10:11:21 | I | Running action 'Transfer' on 'Alice/49717D1246AF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:12:21 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@b1ef49d
 19-05 10:12:21 | I | Time taken = 0:01:00.355565


7221.1684462281.114143<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@b1ef49d

In [9]:
# Now, if we query all nodes

h.query_all_nodes()

 19-05 10:13:20 | I | Result for query=ListInstrument from 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.066062)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,b367c236-10cb-4f76-836a-74e997c48971,Digital Currency INR 500 Note,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'number': 'OMV 336048', 'year': '1981'}"
1,ad3ebdb1-60a5-4d5c-956f-63cb5473ad47,Unique artifact. The Mona Lisa,"CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'holding authority': 'The Louvre Museum, Paris'}"


 19-05 10:13:24 | I | Result for query=ListInstrument from 'Bob/53B3F5FCD707/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.072423)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,b367c236-10cb-4f76-836a-74e997c48971,Digital Currency INR 500 Note,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'number': 'OMV 336048', 'year': '1981'}"


 19-05 10:13:28 | I | Result for query=ListInstrument from 'Charlie/597B2CEE8DB5/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.068591)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,ad3ebdb1-60a5-4d5c-956f-63cb5473ad47,Unique artifact. The Mona Lisa,"CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",None,True,None,True,"{'holding authority': 'The Louvre Museum, Paris'}"


 19-05 10:13:33 | I | Result for query=ListInstrument from 'Alice/49717D1246AF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.073264)


""


In [10]:
# Transfer Example 3 -- Transfer attempt from the Issuer
# Expected -- failed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Authority', 'Charlie')

h.message([req_id, response, return_val])

 19-05 10:13:46 | I | Tranfering instrument id 'b367c236-10cb-4f76-836a-74e997c48971' to 'Charlie'
 19-05 10:13:46 | I | Running action 'Transfer' on 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:13:50 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id b367c236-10cb-4f76-836a-74e997c48971'}
 19-05 10:13:50 | I | Time taken = 0:00:04.056619


7221.1684462426.219317<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id b367c236-10cb-4f76-836a-74e997c48971'}

In [11]:
# Transfer Example 4 -- Transfer attempt from a non-holder Alice
# Expected -- failed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Bob')

h.message([req_id, response, return_val])

 19-05 10:14:22 | I | Tranfering instrument id 'b367c236-10cb-4f76-836a-74e997c48971' to 'Bob'
 19-05 10:14:22 | I | Running action 'Transfer' on 'Alice/49717D1246AF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:14:26 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id b367c236-10cb-4f76-836a-74e997c48971'}
 19-05 10:14:26 | I | Time taken = 0:00:04.058164


7221.1684462462.309547<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id b367c236-10cb-4f76-836a-74e997c48971'}

### Redeem:

* Not all instruments will have expiry/maturity.
* Redeem can be executed by either the owner (holder of the asset) or also by the issuer of the asset (present design, can be controlled)
* If permitted, it will work on the basic nature of the instrument decided right from the issue point(by the issuer).

In Asset type 2, all the redeems apart from non holders should be **succeed**

In [13]:
# Redeem Example 1 -- Redeem by the holder
# Expected -- succeed

h.redeem(return_val_uuid, 'Bob')

 19-05 10:15:47 | I | Redeeming instrument id 'b367c236-10cb-4f76-836a-74e997c48971' quantity 'None'
 19-05 10:15:47 | I | Running action 'Redeem' on 'Bob/53B3F5FCD707/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:16:31 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@2c8b87b5
 19-05 10:16:31 | I | Time taken = 0:00:44.311025


('7221.1684462547.44781',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@2c8b87b5')

In [14]:
# Now, if we query Bob again

h.query('Bob')

""


In [14]:
# Redeem Example 3 -- Partial redeem from Charlie by the Issuer
# Expected -- succeed

h.redeem('ad3ebdb1-60a5-4d5c-956f-63cb5473ad47', 'Authority')

 18-05 23:36:59 | I | Redeeming instrument id 'bf9260e7-98d7-4d97-a298-dc4777e35110' quantity '500'
 18-05 23:36:59 | I | Running action 'Redeem' on 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:37:43 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6183909a
 18-05 23:37:43 | I | Time taken = 0:00:44.320398


('16934.1684449419.211481',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6183909a')

In [15]:
# Now, if we query Bob

h.query('Bob')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,bf9260e7-98d7-4d97-a298-dc4777e35110,eRupee,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",500,True,None,True,{}


In [15]:
# Redeem Example 4 -- Full redeem from Bob by the Issuer
# Expected -- succeed

h.redeem('ad3ebdb1-60a5-4d5c-956f-63cb5473ad47', 'Authority')

 19-05 10:17:12 | I | Redeeming instrument id 'ad3ebdb1-60a5-4d5c-956f-63cb5473ad47' quantity 'None'
 19-05 10:17:12 | I | Running action 'Redeem' on 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:18:00 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@2bc800d0
 19-05 10:18:00 | I | Time taken = 0:00:48.273520


('7221.1684462632.609104',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@2bc800d0')

In [16]:
# Now, if we query Bob again

h.query('Charlie')

""


In [17]:
# Now, if we query all nodes

h.query_all_nodes()

 19-05 10:18:18 | I | Result for query=ListInstrument from 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.093583)


""


 19-05 10:18:22 | I | Result for query=ListInstrument from 'Bob/53B3F5FCD707/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.070018)


""


 19-05 10:18:26 | I | Result for query=ListInstrument from 'Charlie/597B2CEE8DB5/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.073827)


""


 19-05 10:18:30 | I | Result for query=ListInstrument from 'Alice/49717D1246AF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.072198)


""


### Print to PDF:

"Print to PDF" is used for printing any instrument with a specific format.

In [18]:
# Print to PDF Example 1 -- Print the Unique artifacts on Ledger. holding by Alice
## Expected -- succeed

## issue another type-2 asset to Alice for demo purpose
(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Unique artifact. The Mona Lisa', 
                                              quantity=None, transferable=True, expiry=None, verifiable=True, 
                                              attributes={'holding authority': 'The Louvre Museum, Paris'})

pdf_report = h.print_to_pdf('Alice', return_val_uuid, show=False)
webbrowser.open("file:///{}".format(pdf_report))

 19-05 10:20:28 | I | Issuing instrument='Unique artifact. The Mona Lisa' to 'Alice'
 19-05 10:20:28 | I | Running action 'Issue' on 'Authority/F93770F20D7A/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:21:12 | I | 857652d5-a8de-4848-99db-8964536367dc
 19-05 10:21:12 | I | Time taken = 0:00:44.252983
 19-05 10:21:16 | I | http://localhost:9009?data={"name": "Unique artifact. The Mona Lisa", "owner": "CN=Alice, OU=Test Dept, O=R3, L=London, C=GB", "issuer": "CN=Authority, OU=Test Dept, O=R3, L=London, C=GB", "quantity": null, "transferable": true, "expiry": null, "verifiable": true, "attributes": {"holding authority": "The Louvre Museum, Paris"}}
[WDM] - Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.81M/8.81M [00:00<00:00, 23.1MB/s]
 19-05 10:21:21 | I | Time taken = 0:00:08.616605


True

In [19]:
# Print to PDF Example 2 -- Print the Government Bond holding by Issuer
## Expected -- succeed

pdf_report = h.print_to_pdf('Authority', return_val_uuid, show=False)
webbrowser.open("file:///{}".format(pdf_report))

 19-05 10:21:49 | I | http://localhost:9009?data={"name": "Unique artifact. The Mona Lisa", "owner": "CN=Alice, OU=Test Dept, O=R3, L=London, C=GB", "issuer": "CN=Authority, OU=Test Dept, O=R3, L=London, C=GB", "quantity": null, "transferable": true, "expiry": null, "verifiable": true, "attributes": {"holding authority": "The Louvre Museum, Paris"}}
 19-05 10:21:52 | I | Time taken = 0:00:07.639856


True

In [20]:
# Print to PDF Example 3 -- Print the Government Bond holding by non holder
## Expected -- failed

pdf_report = h.print_to_pdf('Charlie',return_val_uuid, show=False)
#webbrowser.open("file:///{}".format(pdf_report))

 19-05 10:22:00 | E | No data matching id='857652d5-a8de-4848-99db-8964536367dc' in 'Charlie'
0:28: execution error: File some object wasn’t found. (-43)


True